In [14]:
import json
import pandas as pd

In [15]:
#Find option columns from dataframe
'''
df = pd.read_csv("dataset_after_encoding.csv")
options_columns = [col for col in df.columns.values if df[col].nunique() <=3]
'''

'\ndf = pd.read_csv("dataset_after_encoding.csv")\noptions_columns = [col for col in df.columns.values if df[col].nunique() <=3]\n'

In [16]:
with open("option_columns.json","r") as f:
    option_columns = json.load(f)

In [17]:
df = pd.read_csv("dataset_after_encoding.csv", dtype={k:"int8" for k in option_columns})

In [18]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69362 entries, 0 to 69361
Columns: 12654 entries, X86_LOCAL_APIC to cid
dtypes: float64(6), int64(37), int8(12611)
memory usage: 857.0 MB


In [20]:
df.query("compile_success == False").shape

(3507, 12654)

In [22]:
#df.drop("Unnamed: 0", inplace=True, axis=1)

In [23]:
basic_head = ["cid", "time", "date"] # "compile"
compilation_status_column_name = 'compile_success'
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

In [93]:
from sklearn.model_selection import train_test_split
from sklearn import tree


TESTING_SIZE= 0.001 #0.99 # 0.001 # 0.9
X_train, X_test, y_train, y_test = train_test_split(
    df
    .drop(columns=["cid"])
    .drop(columns=size_methods)
    .drop(columns=compilation_status_column_name), 
    df[compilation_status_column_name], test_size=TESTING_SIZE, random_state=0)  
clf = tree.DecisionTreeClassifier() #GradientBoostingClassifier(n_estimators=100) #RandomForestRegressor(n_estimators=100) #   #GradientBoostingRegressor(n_estimators=100)  
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [94]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score

y_pred = clf.predict(X_test)

acc = accuracy_score (y_test, y_pred)
prec = precision_score (y_test, y_pred)
reca = recall_score (y_test, y_pred)
f1 = f1_score (y_test, y_pred)
balance_acc = balanced_accuracy_score (y_test, y_pred)
print("Accuracy score:", acc)
print("Precision score:", prec)
print("Recall score:", reca)
print("F1 score:", f1)
print("Balance accuracy score:", balance_acc)

Accuracy score: 1.0
Precision score: 1.0
Recall score: 1.0
F1 score: 1.0
Balance accuracy score: 1.0


In [95]:
import graphviz 

def printTree(clf, feature_names):
    dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=feature_names,  
                         filled=True, rounded=True,
                         special_characters=True)  
    graph = graphviz.Source(dot_data)  
    graph.render("TUXML_compilation_failures")
    
printTree(clf, df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=compilation_status_column_name).columns)

In [96]:
from sklearn.tree import _tree

def tree_to_rules(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    #print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, previous_rules):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            #print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], previous_rules+[name + " <= " + str(threshold)])
            #print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], previous_rules+[name + " > " + str(threshold)])
        else:
            print(" & ".join(previous_rules) + " ---> " + str(tree_.value[node]))

    recurse(0, [])
    
tree_to_rules(clf, df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=compilation_status_column_name).columns)

AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & MLX5_INFINIBAND <= 0.5 & ACPI_BUTTON <= 0.5 ---> [[1. 0.]]
AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & MLX5_INFINIBAND <= 0.5 & ACPI_BUTTON > 0.5 ---> [[0. 1.]]
AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & MLX5_INFINIBAND > 0.5 & ISDN_DIVAS_USERIDI <= 0.5 & PVPANIC <= 0.5 ---> [[1. 0.]]
AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & MLX5_INFINIBAND > 0.5 & ISDN_DIVAS_USERIDI <= 0.5 & PVPANIC > 0.5 ---> [[0. 3.]]
AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & 

In [97]:
def tree_to_failure_rules(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    #print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, previous_rules):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            #print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], previous_rules+[name + " <= " + str(threshold)])
            #print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], previous_rules+[name + " > " + str(threshold)])
        else:
            if tree_.value[node][0][0] > tree_.value[node][0][1]:
                print(" & ".join(previous_rules) + " ---> " + str(tree_.value[node]))

    recurse(0, [])
    
tree_to_failure_rules(clf, df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=compilation_status_column_name).columns)

AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & MLX5_INFINIBAND <= 0.5 & ACPI_BUTTON <= 0.5 ---> [[1. 0.]]
AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & MLX5_INFINIBAND > 0.5 & ISDN_DIVAS_USERIDI <= 0.5 & PVPANIC <= 0.5 ---> [[1. 0.]]
AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & MLX5_INFINIBAND > 0.5 & ISDN_DIVAS_USERIDI > 0.5 & INFINIBAND_SRPT <= 1.5 & WIL6210_TRACING <= 0.5 & XEN_PCIDEV_BACKEND <= 1.5 & FB_GEODE_GX1 <= 0.5 & ISL29125 > 1.5 ---> [[1. 0.]]
AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO <= 1.5 & MLX5_INFINIBAND > 0.5 & ISDN_DIVAS_USERIDI > 0.5 & INFINIBAND_SRPT <= 1.5 & WIL6210_TRACING <= 0.5 & X

In [98]:
def find_rule_for_configuration(tree, feature_names, configuration):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    def recurse(node, previous_rules, configuration):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            #print ("{}if {} <= {}:".format(indent, name, threshold))
            if configuration[name] <= threshold:
                recurse(tree_.children_left[node], previous_rules+[name + " <= " + str(threshold)], configuration)
            #print ("{}else:  # if {} > {}".format(indent, name, threshold))
            else:
                recurse(tree_.children_right[node], previous_rules+[name + " > " + str(threshold)], configuration)
        else:
            #if tree_.value[node][0][0] > tree_.value[node][0][1]:
            print(" & ".join(previous_rules))
    
    recurse(0, [], configuration)
    
find_rule_for_configuration(clf, df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=compilation_status_column_name).columns, df.loc[65586]) # <--give the row you want the rules for

AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR <= 0.5 & DRM_VBOXVIDEO > 1.5


In [99]:
find_rule_for_configuration(clf, df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=compilation_status_column_name).columns, df.query("cid == 92072").iloc[0]) 

AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR > 0.5 & UBSAN_SANITIZE_ALL <= 0.5 & VIDEO_XILINX_TPG > 0.5 & CRC32 > 1.0 & VIDEO_ATOMISP > 0.5 & EDAC_DECODE_MCE > 0.5 & RSI_SDIO > 0.5 & SPEAKUP_SYNTH_BNS <= 0.5 & SND_SOC_INTEL_SKL_NAU88L25_SSM4567_MACH <= 1.5 & SND_PDAUDIOCF <= 1.5 & TYPEC_WCOVE <= 1.5 & CAN_GS_USB <= 1.5 & SND_ATMEL_SOC > 1.5 & REGULATOR_DA9062 > 1.5


In [100]:
find_rule_for_configuration(clf, df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=compilation_status_column_name).columns, df.query("cid == 84768").iloc[0]) # 

AIC79XX_BUILD_FIRMWARE <= 0.5 & AIC7XXX_BUILD_FIRMWARE <= 0.5 & WANXL_BUILD_FIRMWARE <= 0.5 & GENERIC_ALLOCATOR > 0.5 & UBSAN_SANITIZE_ALL <= 0.5 & VIDEO_XILINX_TPG > 0.5 & CRC32 > 1.0 & VIDEO_ATOMISP > 0.5 & EDAC_DECODE_MCE > 0.5 & RSI_SDIO <= 0.5 & SND_SOC_WM8770 > 1.5


In [101]:
df.query("AIC7XXX_BUILD_FIRMWARE == 0 & AIC79XX_BUILD_FIRMWARE == 0 & WANXL_BUILD_FIRMWARE == 1 & compile_success == False").shape

(419, 12654)

In [102]:
for cid in df.query("AIC7XXX_BUILD_FIRMWARE == 1 & AIC79XX_BUILD_FIRMWARE == 1 & WANXL_BUILD_FIRMWARE == 1 & compile_success == False")['cid']:
    find_rule_for_configuration(clf, df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=compilation_status_column_name).columns, df.query("cid == " + str(cid)).iloc[0]) # 

AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5
AIC79XX_BUILD_FIRMWARE > 0.5


In [103]:
df.query("GENERIC_ALLOCATOR == 0 & DRM_VBOXVIDEO == 2 & (AIC7XXX_BUILD_FIRMWARE == 1 | AIC79XX_BUILD_FIRMWARE == 1 | WANXL_BUILD_FIRMWARE == 1)  & compile_success == False").shape

(13, 12654)

In [104]:
df.query("cid == 92072")

,X86_LOCAL_APIC,OPENVSWITCH,TEXTSEARCH_FSM,LOCKDEP_SUPPORT,GENERIC_CLOCKEVENTS_MIN_ADJUST,NETFILTER_XT_MATCH_TCPMSS,MPLS,NFC_HCI,STACKTRACE_SUPPORT,GENERIC_CMOS_UPDATE,...,XZ-vmlinux,XZ,LZO-bzImage,LZO-vmlinux,LZO,LZ4-bzImage,LZ4-vmlinux,LZ4,compile_success,cid
61435,0,1,1,0,0,1,0,1,0,0,...,-1,-1,-1.0,-1,-1,-1.0,-1,-1,0,92072
